In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [13]:
import malaya_speech
import malaya_speech.train.model.demucs as demucs
import tensorflow as tf
import numpy as np

In [4]:
i = tf.placeholder(tf.float32, [None, 1])
model = demucs.Model(i, logging = True)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Tensor("Pad_1:0", shape=(?, 99668, 1), dtype=float32)
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use `tf.keras.layers.Conv1D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Tensor("sequential/conv1d_1/mul:0", shape=(?, 24916, 64), dtype=float32)
Tensor("sequential_2/conv1d_4/mul:0", shape=(?, 6228, 128), dtype=float32)
Tensor("sequential_4/conv1d_7/mul:0", shape=(?, 1556, 256), dtype=float32)
Tensor("sequential_6/conv1d_10/mul:0", shape=(?, 388, 512), dtype=float32)
Tensor("sequential_8/conv1d_13/mul:0", shape=(?, 96, 1024), dtype=float32)
x 

In [5]:
model.logits

<tf.Tensor 'strided_slice_21:0' shape=(?, 4) dtype=float32>

In [11]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [10]:
y, sr = malaya_speech.load('../speech/7021-79759-0004.flac', sr = 44100)
y = y[:sr * 10]
len(y), len(y) / sr

(441000, 10.0)

In [14]:
%%time

r = sess.run(model.logits, feed_dict = {i: np.expand_dims(y, -1)})

CPU times: user 21.1 s, sys: 9.14 s, total: 30.2 s
Wall time: 6.28 s


In [16]:
r.shape

(441000, 4)